In [2]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import train_test_split
import clang.cindex
import tempfile
from sklearn.model_selection import KFold



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [41]:

data = pd.read_csv('Data_Ast.csv')
data.head(5)

,Unnamed: 0,Question,Correct_Code,Code_with_Error,Total_Marks,AST_full
0,0,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,7.0,CursorKind.FUNCTION_DECL printFactors\n Curso...
1,1,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,8.0,CursorKind.FUNCTION_DECL printFactors\n Curso...
2,2,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,5.0,CursorKind.FUNCTION_DECL printFactors\n Curso...
3,3,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\n\nvoid printFactors(int nu...,7.0,CursorKind.FUNCTION_DECL printFactors\n Curso...
4,4,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\n\nvoid printFactors(int nu...,5.0,CursorKind.FUNCTION_DECL printFactors\n Curso...


In [38]:
data.head()

,Unnamed: 0,Question,Correct_Code,Code_with_Error,Total_Marks,AST_full,AST_predicted_values_Random Forest,AST_predicted_values_XGBoost,AST_predicted_values_CatBoost,AST_predicted_values_SVR,AST_predicted_values_KNN
0,0,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,7.0,CursorKind.FUNCTION_DECL printFactors\n Curso...,7.780476,9.158783,8.083245,5.911610,7.2
1,1,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,8.0,CursorKind.FUNCTION_DECL printFactors\n Curso...,5.739167,6.029593,5.678359,6.269147,5.6
2,2,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,5.0,CursorKind.FUNCTION_DECL printFactors\n Curso...,5.064786,5.561528,4.966199,6.273282,7.2
3,3,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\n\nvoid printFactors(int nu...,7.0,CursorKind.FUNCTION_DECL printFactors\n Curso...,7.174000,6.985331,6.514410,6.105001,7.2
4,4,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\n\nvoid printFactors(int nu...,5.0,CursorKind.FUNCTION_DECL printFactors\n Curso...,6.045714,6.096070,5.765330,6.329648,5.0


In [42]:
Indexes =  [526, 529, 530, 532, 533, 534, 693, 694, 845, 848, 853, 858, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874]
data = data.drop(index=Indexes).reset_index(drop=True)

In [8]:
ast = torch.load('batch_0.pt')

In [9]:
ast.shape

torch.Size([100, 5, 512, 768])

In [4]:
import torch

file_names = [f'batch_{i}.pt' for i in range(10)]  # batch_0.pt, batch_1.pt, ..., batch_8.pt

# Initialize an empty list to store the loaded tensors
all_tensors = []

# Load and append each tensor to the list
for file_name in file_names:
    tensor = torch.load(file_name)  # Load the tensor from the file
    all_tensors.append(tensor)  # Append it to the list

# Concatenate all tensors along the batch dimension (assuming they're of the same shape)
combined_tensor = torch.cat(all_tensors, dim=0)  # dim=0 means concatenating along the batch dimension

# Save the combined tensor to a new .pt file
torch.save(combined_tensor, 'combined_batches.pt')

print(f"Combined tensor saved as 'combined_batches.pt' with shape: {combined_tensor.shape}")


Combined tensor saved as 'combined_batches.pt' with shape: torch.Size([973, 5, 512, 768])


In [5]:


# Extract the CLS token (first token from the sequence, i.e., index 0 along dimension 2)
cls_tokens = combined_tensor[:, :, 0, :]  # Shape: [100, 5, 768]

print(f"CLS tokens shape: {cls_tokens.shape}")


CLS tokens shape: torch.Size([973, 5, 768])


## BiLSTM + Neural Nets linear regression layer

In [44]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd

# Define your device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load embeddings
# combined_batches = torch.load('combined_batches.pt', map_location=device)  # Shape: [973, 5, 512, 768]
combined_embeddings_ = combined_tensor[:, :, 0, :]  # Extract CLS token (dim=2), Shape: [973, 5, 768]
cls_embeddings = combined_embeddings_.mean(dim=1)  # Mean pooling across 5 batches, Shape: [973, 768]
print(f"Embeddings loaded. Shape: {cls_embeddings.shape}")

labels = torch.tensor(data['Total_Marks']).float()  # Assuming 'Total_Marks' column contains the target labels
assert len(labels) == cls_embeddings.size(0), "Number of labels must match number of embeddings"

# Normalize labels (min-max normalization)
labels_min = labels.min()
labels_max = labels.max()
labels = (labels - labels_min) / (labels_max - labels_min)  # Normalize to range [0, 1]

# Custom Dataset for embeddings and labels
class CodeDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

# BiLSTM Model for regression with an extra hidden layer
class BiLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(BiLSTMModel, self).__init__()
        self.bilstm = nn.LSTM(
            input_dim,
            hidden_dim,
            num_layers,
            batch_first=True,
            bidirectional=True
        )
        self.fc1 = nn.Linear(hidden_dim * 2, 128)  # Additional hidden layer
        self.fc2 = nn.Linear(128, 1)  # Output layer for regression

    def forward(self, x):
        lstm_out, _ = self.bilstm(x)  # [batch_size, seq_len, 2*hidden_dim]
        lstm_out = lstm_out.mean(dim=1)  # Mean pooling over sequence length
        fc1_out = torch.relu(self.fc1(lstm_out))  # First hidden layer
        output = self.fc2(fc1_out)  # Output layer
        return output

# Hyperparameters
input_dim = cls_embeddings.size(-1)  # 768
hidden_dim = 256  # Hidden state size
num_layers = 2   # Number of BiLSTM layers
batch_size = 32  # Batch size for training
num_epochs = 100  # Number of epochs
learning_rate = 0.001

# Initialize the model, loss function, and optimizer
criterion = nn.MSELoss()

# 10-Fold Cross-Validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
predicted_values = np.zeros(len(labels))  # Store predictions for all folds

for fold, (train_idx, test_idx) in enumerate(kf.split(cls_embeddings)):
    print(f"Fold {fold + 1}")

    # Prepare data
    train_dataset = CodeDataset(cls_embeddings[train_idx], labels[train_idx])
    test_dataset = CodeDataset(cls_embeddings[test_idx], labels[test_idx])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Reset model and optimizer
    model = BiLSTMModel(input_dim=input_dim, hidden_dim=hidden_dim, num_layers=num_layers).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0

        for inputs, target in train_loader:
            inputs = inputs.to(device).float().unsqueeze(1)  # Add sequence length dimension
            target = target.to(device).float()

            # Forward pass
            output = model(inputs).squeeze(-1)  # [batch_size]
            loss = criterion(output, target)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

    # Evaluate the model
    model.eval()
    test_preds = []
    test_targets = []

    with torch.no_grad():
        for inputs, target in test_loader:
            inputs = inputs.to(device).float().unsqueeze(1)
            target = target.to(device).float()

            output = model(inputs).squeeze(-1)  # [batch_size]
            test_preds.extend(output.cpu().numpy())
            test_targets.extend(target.cpu().numpy())

    # De-normalize predictions and targets
    test_preds = np.array(test_preds) * (labels_max - labels_min).item() + labels_min.item()
    test_targets = np.array(test_targets) * (labels_max - labels_min).item() + labels_min.item()

    # Store predictions back into the dataset
    data.loc[test_idx, 'Predictions'] = test_preds

    # Store predictions in their respective indices for metrics calculation
    predicted_values[test_idx] = test_preds

    # Calculate metrics
    r2 = r2_score(test_targets, test_preds)
    rmse = mean_squared_error(test_targets, test_preds, squared=False)
    mape = mean_absolute_percentage_error(test_targets, test_preds)

    print(f"Fold {fold + 1} R²: {r2:.4f}, RMSE: {rmse:.4f}, MAPE: {mape:.4f}")

# Save the predicted values back to 'data'
data.to_csv('data_with_predictions.csv', index=False)
print("Predictions saved to 'data_with_predictions.csv'!")

# Final metrics on the entire dataset
r2_final = r2_score(labels * (labels_max - labels_min).item() + labels_min.item(), predicted_values)
rmse_final = mean_squared_error(labels * (labels_max - labels_min).item() + labels_min.item(), predicted_values, squared=False)
mape_final = mean_absolute_percentage_error(labels * (labels_max - labels_min).item() + labels_min.item(), predicted_values)

print(f"Final Metrics - R²: {r2_final:.4f}, RMSE: {rmse_final:.4f}, MAPE: {mape_final:.4f}")


Embeddings loaded. Shape: torch.Size([973, 768])
Fold 1
Epoch 1/100, Loss: 3.2354
Epoch 2/100, Loss: 1.3965
Epoch 3/100, Loss: 1.3665
Epoch 4/100, Loss: 1.2825
Epoch 5/100, Loss: 1.1965
Epoch 6/100, Loss: 1.2481
Epoch 7/100, Loss: 1.1626
Epoch 8/100, Loss: 1.1549
Epoch 9/100, Loss: 1.1380
Epoch 10/100, Loss: 1.0893
Epoch 11/100, Loss: 1.0724
Epoch 12/100, Loss: 1.0293
Epoch 13/100, Loss: 0.9813
Epoch 14/100, Loss: 1.0171
Epoch 15/100, Loss: 0.9670
Epoch 16/100, Loss: 0.9318
Epoch 17/100, Loss: 0.9158
Epoch 18/100, Loss: 0.9983
Epoch 19/100, Loss: 0.9989
Epoch 20/100, Loss: 0.9211
Epoch 21/100, Loss: 0.8846
Epoch 22/100, Loss: 0.8875
Epoch 23/100, Loss: 0.8855
Epoch 24/100, Loss: 0.8379
Epoch 25/100, Loss: 0.8323
Epoch 26/100, Loss: 0.8296
Epoch 27/100, Loss: 0.8055
Epoch 28/100, Loss: 0.7992
Epoch 29/100, Loss: 0.8672
Epoch 30/100, Loss: 0.8434
Epoch 31/100, Loss: 0.8018
Epoch 32/100, Loss: 0.8040
Epoch 33/100, Loss: 0.7676
Epoch 34/100, Loss: 0.7329
Epoch 35/100, Loss: 0.7682
Epoch 36

/Users/jagadeeshgurram/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Epoch 1/100, Loss: 3.9405
Epoch 2/100, Loss: 1.4375
Epoch 3/100, Loss: 1.2799
Epoch 4/100, Loss: 1.2531
Epoch 5/100, Loss: 1.1997
Epoch 6/100, Loss: 1.2087
Epoch 7/100, Loss: 1.1147
Epoch 8/100, Loss: 1.1536
Epoch 9/100, Loss: 1.0298
Epoch 10/100, Loss: 0.9896
Epoch 11/100, Loss: 1.0303
Epoch 12/100, Loss: 1.0007
Epoch 13/100, Loss: 0.9424
Epoch 14/100, Loss: 0.9175
Epoch 15/100, Loss: 0.9309
Epoch 16/100, Loss: 0.8980
Epoch 17/100, Loss: 0.8775
Epoch 18/100, Loss: 0.8497
Epoch 19/100, Loss: 0.8582
Epoch 20/100, Loss: 0.8548
Epoch 21/100, Loss: 0.8725
Epoch 22/100, Loss: 0.8353
Epoch 23/100, Loss: 0.8215
Epoch 24/100, Loss: 0.8016
Epoch 25/100, Loss: 0.8167
Epoch 26/100, Loss: 0.8983
Epoch 27/100, Loss: 0.8267
Epoch 28/100, Loss: 0.7602
Epoch 29/100, Loss: 0.7338
Epoch 30/100, Loss: 0.7362
Epoch 31/100, Loss: 0.7491
Epoch 32/100, Loss: 0.7996
Epoch 33/100, Loss: 0.7576
Epoch 34/100, Loss: 0.7038
Epoch 35/100, Loss: 0.7041
Epoch 36/100, Loss: 0.6961
Epoch 37/100, Loss: 0.6843
Epoch 38/1

/Users/jagadeeshgurram/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Epoch 1/100, Loss: 3.8067
Epoch 2/100, Loss: 1.3950
Epoch 3/100, Loss: 1.3437
Epoch 4/100, Loss: 1.2987
Epoch 5/100, Loss: 1.2787
Epoch 6/100, Loss: 1.2167
Epoch 7/100, Loss: 1.1649
Epoch 8/100, Loss: 1.1814
Epoch 9/100, Loss: 1.0857
Epoch 10/100, Loss: 1.0255
Epoch 11/100, Loss: 1.1119
Epoch 12/100, Loss: 0.9973
Epoch 13/100, Loss: 0.9960
Epoch 14/100, Loss: 0.9884
Epoch 15/100, Loss: 0.9961
Epoch 16/100, Loss: 0.9720
Epoch 17/100, Loss: 0.9089
Epoch 18/100, Loss: 0.9363
Epoch 19/100, Loss: 0.9535
Epoch 20/100, Loss: 0.8994
Epoch 21/100, Loss: 0.8976
Epoch 22/100, Loss: 0.8588
Epoch 23/100, Loss: 0.8524
Epoch 24/100, Loss: 0.9029
Epoch 25/100, Loss: 0.8425
Epoch 26/100, Loss: 0.8489
Epoch 27/100, Loss: 0.8565
Epoch 28/100, Loss: 0.8397
Epoch 29/100, Loss: 0.8273
Epoch 30/100, Loss: 0.8105
Epoch 31/100, Loss: 0.8218
Epoch 32/100, Loss: 0.8550
Epoch 33/100, Loss: 0.8002
Epoch 34/100, Loss: 0.7552
Epoch 35/100, Loss: 0.7393
Epoch 36/100, Loss: 0.7447
Epoch 37/100, Loss: 0.7512
Epoch 38/1

...

/Users/jagadeeshgurram/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Epoch 1/100, Loss: 4.3042
Epoch 2/100, Loss: 1.4264
Epoch 3/100, Loss: 1.3343
Epoch 4/100, Loss: 1.3033
Epoch 5/100, Loss: 1.2016
Epoch 6/100, Loss: 1.2189
Epoch 7/100, Loss: 1.1310
Epoch 8/100, Loss: 1.0686
Epoch 9/100, Loss: 1.0367
Epoch 10/100, Loss: 1.0053
Epoch 11/100, Loss: 1.0487
Epoch 12/100, Loss: 1.0417
Epoch 13/100, Loss: 1.0098
Epoch 14/100, Loss: 0.9793
Epoch 15/100, Loss: 0.9354
Epoch 16/100, Loss: 0.9053
Epoch 17/100, Loss: 0.9239
Epoch 18/100, Loss: 0.9449
Epoch 19/100, Loss: 0.9195
Epoch 20/100, Loss: 0.8794
Epoch 21/100, Loss: 0.8677
Epoch 22/100, Loss: 0.8415
Epoch 23/100, Loss: 0.8462
Epoch 24/100, Loss: 0.8349
Epoch 25/100, Loss: 0.8034
Epoch 26/100, Loss: 0.7885
Epoch 27/100, Loss: 0.7867
Epoch 28/100, Loss: 0.8057
Epoch 29/100, Loss: 0.7867
Epoch 30/100, Loss: 0.7742
Epoch 31/100, Loss: 0.7439
Epoch 32/100, Loss: 0.7267
Epoch 33/100, Loss: 0.7441
Epoch 34/100, Loss: 0.7349
Epoch 35/100, Loss: 0.7094
Epoch 36/100, Loss: 0.7067
Epoch 37/100, Loss: 0.7248
Epoch 38/1

/Users/jagadeeshgurram/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Epoch 1/100, Loss: 4.0255
Epoch 2/100, Loss: 1.4648
Epoch 3/100, Loss: 1.4307
Epoch 4/100, Loss: 1.3105
Epoch 5/100, Loss: 1.2428
Epoch 6/100, Loss: 1.2070
Epoch 7/100, Loss: 1.1525
Epoch 8/100, Loss: 1.1696
Epoch 9/100, Loss: 1.1736
Epoch 10/100, Loss: 1.0861
Epoch 11/100, Loss: 1.0716
Epoch 12/100, Loss: 1.1237
Epoch 13/100, Loss: 1.0177
Epoch 14/100, Loss: 1.0042
Epoch 15/100, Loss: 1.0506
Epoch 16/100, Loss: 0.9488
Epoch 17/100, Loss: 0.9988
Epoch 18/100, Loss: 0.9708
Epoch 19/100, Loss: 0.9061
Epoch 20/100, Loss: 0.9115
Epoch 21/100, Loss: 0.9435
Epoch 22/100, Loss: 0.8599
Epoch 23/100, Loss: 0.8491
Epoch 24/100, Loss: 0.8346
Epoch 25/100, Loss: 0.8681
Epoch 26/100, Loss: 0.8524
Epoch 27/100, Loss: 0.8489
Epoch 28/100, Loss: 0.8359
Epoch 29/100, Loss: 0.8177
Epoch 30/100, Loss: 0.7655
Epoch 31/100, Loss: 0.7906
Epoch 32/100, Loss: 0.8032
Epoch 33/100, Loss: 0.7784
Epoch 34/100, Loss: 0.7766
Epoch 35/100, Loss: 0.7177
Epoch 36/100, Loss: 0.7370
Epoch 37/100, Loss: 0.7288
Epoch 38/1

/Users/jagadeeshgurram/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/jagadeeshgurram/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [45]:
data.head()

,Unnamed: 0,Question,Correct_Code,Code_with_Error,Total_Marks,AST_full,Predictions
0,0,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,7.0,CursorKind.FUNCTION_DECL printFactors\n Curso...,6.941752
1,1,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,8.0,CursorKind.FUNCTION_DECL printFactors\n Curso...,5.128729
2,2,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,5.0,CursorKind.FUNCTION_DECL printFactors\n Curso...,4.312079
3,3,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\n\nvoid printFactors(int nu...,7.0,CursorKind.FUNCTION_DECL printFactors\n Curso...,6.914923
4,4,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\n\nvoid printFactors(int nu...,5.0,CursorKind.FUNCTION_DECL printFactors\n Curso...,7.299567


In [50]:
mean_squared_error(data['Total_Marks'], data['Predictions'],squared=False)

/Users/jagadeeshgurram/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


2.2363955809803833

## BiLSTM

In [23]:
import torch
import torch.nn as nn


# Define BiLSTM model
class BiLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(BiLSTMModel, self).__init__()
        self.bilstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)

    def forward(self, x):
        lstm_out, _ = self.bilstm(x)  # Output: [batch_size, seq_len, hidden_dim * 2]
        return lstm_out

# Parameters
input_dim = cls_tokens.shape[-1]  # 768
hidden_dim = 256
num_layers = 2

# Initialize and process through BiLSTM
bilstm_model = BiLSTMModel(input_dim=input_dim, hidden_dim=hidden_dim, num_layers=num_layers)
bilstm_out = bilstm_model(cls_tokens)  # Input shape: [100, 5, 768]

print(f"BiLSTM output shape: {bilstm_out.shape}")  # Expected: [100, 5, 256]


BiLSTM output shape: torch.Size([973, 5, 512])


In [21]:
print(bilstm_out)


tensor([[[-7.9638e-02,  4.3596e-02,  3.7416e-02,  ..., -1.6467e-03,
          -1.8959e-02,  1.8841e-02],
         [-7.0362e-02,  1.1500e-01,  4.4823e-02,  ...,  2.4961e-03,
          -1.3251e-02,  1.6140e-02],
         [-6.7673e-02,  1.0427e-01,  5.5073e-02,  ..., -2.8472e-02,
          -2.1572e-02,  6.8984e-03],
         [-7.2155e-02,  8.8017e-02,  6.2113e-02,  ..., -3.0757e-02,
          -1.8411e-02,  1.1647e-02],
         [-7.2060e-02,  7.2969e-02,  7.0614e-02,  ..., -2.6238e-02,
          -1.4617e-02,  1.3848e-02]],

        [[-8.8478e-02,  2.6850e-02,  5.0853e-02,  ..., -8.5485e-02,
           9.4792e-03,  6.5808e-03],
         [-9.0818e-02,  3.1558e-02,  6.3055e-02,  ..., -6.6047e-02,
           3.0052e-02, -8.8734e-03],
         [-8.1569e-02,  4.0304e-02,  6.8501e-02,  ..., -2.4639e-02,
           8.2662e-03,  5.4836e-04],
         [-7.6553e-02,  4.6134e-02,  6.8535e-02,  ..., -3.0234e-02,
           6.9513e-05,  9.3080e-03],
         [-7.1653e-02,  4.6879e-02,  7.1848e-02,  ...

## Regression Models

In [29]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import KFold

# Example: Assuming `bilstm_out` is already defined with shape (973, 5, 512)
# Flattening the output to (973, 5 * 512)
bilstm_out_flattened = bilstm_out.detach().reshape(bilstm_out.size(0), -1).numpy()  # Shape: (973, 2560)
# Assuming you have a column `Total_Marks` in your filtered_data for actual values
actual_values = data["Total_Marks"]

# Cross-validation setup
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Initialize models
models = {
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=200, max_depth=8, learning_rate=0.1, random_state=42),
    "CatBoost": CatBoostRegressor(iterations=500, learning_rate=0.1, depth=8, verbose=0),
    "SVR": SVR(),
    "KNN": KNeighborsRegressor(n_neighbors=5)
}

# Dictionary to store predicted values for each model
predicted_values = {
    "Random Forest": np.zeros(len(actual_values)),
    "XGBoost": np.zeros(len(actual_values)),
    "CatBoost": np.zeros(len(actual_values)),
    "SVR": np.zeros(len(actual_values)),
    "KNN": np.zeros(len(actual_values))
}

# Store metrics
metrics = {
    "Model": [],
    "MAPE": [],
    "R²": [],
    "RMSE": [],
}

# Loop over each model to train and evaluate
for model_name, model in models.items():
    print(f"\nTraining {model_name} model...")
    
    # Initialize lists to store metrics for each fold
    fold_metrics = {
        "R²": [],
        "RMSE": [],
        "MAPE": []
    }
    
    # Perform 10-fold CV
    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(bilstm_out_flattened)):
        print(f"\nProcessing Fold {fold_idx + 1}...")
        
        # Split data
        X_train, X_test = bilstm_out_flattened[train_idx], bilstm_out_flattened[test_idx]
        y_train, y_test = actual_values[train_idx], actual_values[test_idx]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Predict on test set
        y_pred = model.predict(X_test)
        predicted_values[model_name][test_idx] = y_pred  # Store predictions
        
        # Calculate metrics
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mape = mean_absolute_percentage_error(y_test, y_pred)
        
        # Store fold metrics
        fold_metrics["R²"].append(r2)
        fold_metrics["RMSE"].append(rmse)
        fold_metrics["MAPE"].append(mape)
        
        print(f"Fold {fold_idx + 1} Results: R² = {r2:.4f}, RMSE = {rmse:.4f}, MAPE = {mape:.4f}")

    # Calculate average metrics across folds
    avg_r2 = np.mean(fold_metrics["R²"])
    avg_rmse = np.mean(fold_metrics["RMSE"])
    avg_mape = np.mean(fold_metrics["MAPE"])

    # Store the model results
    metrics["Model"].append(model_name)
    metrics["MAPE"].append(avg_mape)
    metrics["R²"].append(avg_r2)
    metrics["RMSE"].append(avg_rmse)

    print(f"\n{model_name} - Average Results: R² = {avg_r2:.4f}, RMSE = {avg_rmse:.4f}, MAPE = {avg_mape:.4f}")

# Convert metrics into DataFrame and save it
scores_df = pd.DataFrame(metrics)
csv_filename = "AST_model_evaluation_scores.csv"
scores_df.to_csv(csv_filename, index=False)
print(f"Evaluation scores saved to {csv_filename}")





Training Random Forest model...

Processing Fold 1...
Fold 1 Results: R² = 0.3483, RMSE = 1.9051, MAPE = 123374436029899.4688

Processing Fold 2...
Fold 2 Results: R² = 0.1843, RMSE = 2.1504, MAPE = 0.4048

Processing Fold 3...
Fold 3 Results: R² = 0.2738, RMSE = 2.0334, MAPE = 0.4206

Processing Fold 4...
Fold 4 Results: R² = 0.0673, RMSE = 2.0666, MAPE = 0.3431

Processing Fold 5...
Fold 5 Results: R² = 0.2262, RMSE = 1.9157, MAPE = 191774415060349.1250

Processing Fold 6...
Fold 6 Results: R² = 0.3519, RMSE = 1.8520, MAPE = 0.4411

Processing Fold 7...
Fold 7 Results: R² = 0.1583, RMSE = 1.9853, MAPE = 0.3227

Processing Fold 8...
Fold 8 Results: R² = 0.1970, RMSE = 1.8349, MAPE = 0.3054

Processing Fold 9...
Fold 9 Results: R² = 0.2675, RMSE = 2.0477, MAPE = 680527730438096.8750

Processing Fold 10...
Fold 10 Results: R² = 0.0443, RMSE = 2.2601, MAPE = 223694810504663.7500

Random Forest - Average Results: R² = 0.2119, RMSE = 2.0051, MAPE = 121937139203301.1562

Training XGBoost m

NameError: name 'filtered_data' is not defined

In [31]:
# Save predicted values back to the dataset with appropriate column names
for model_name, predictions in predicted_values.items():
    data[f"AST_predicted_values_{model_name}"] = predictions

# Save the dataset with predictions
predictions_filename = "dataset_with_AST_predictions.csv"
data.to_csv(predictions_filename, index=False)
print(f"Predictions saved to {predictions_filename}")

Predictions saved to dataset_with_AST_predictions.csv


## Evaluation Metrics

In [33]:
data = data[data['Total_Marks']>0]

In [35]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import numpy as np

# Load the dataset if not already loaded
# Assuming `filtered_data` is a DataFrame containing the relevant data
# filtered_data = pd.read_csv("dataset_with_AST_predictions.csv")

# Extract columns for actual and predicted values
actual_values = data["Total_Marks"]
predicted_columns = {
    "Random Forest": data["AST_predicted_values_Random Forest"],
    "XGBoost": data["AST_predicted_values_XGBoost"],
    "CatBoost": data["AST_predicted_values_CatBoost"],
    "SVR": data["AST_predicted_values_SVR"],
    "KNN": data["AST_predicted_values_KNN"],
}

# Initialize lists to store scores
results = {
    "Model": [],
    "MAPE": [],
    "R²": [],
    "RMSE": [],
}

# Calculate metrics for each model
for model_name, predictions in predicted_columns.items():
    mape = mean_absolute_percentage_error(actual_values, predictions)
    r2 = r2_score(actual_values, predictions)
    rmse = np.sqrt(mean_squared_error(actual_values, predictions))
    
    # Append results
    results["Model"].append(model_name)
    results["MAPE"].append(mape)
    results["R²"].append(r2)
    results["RMSE"].append(rmse)

# Create a DataFrame for the results
scores_df = pd.DataFrame(results)

# Save to CSV
csv_filename = "AST_model_evaluation_scores.csv"

scores_df.to_csv(csv_filename, index=False)

print(f"Evaluation scores saved to {csv_filename}")

# Print the results DataFrame for reference
print(scores_df)


Evaluation scores saved to AST_model_evaluation_scores.csv
           Model      MAPE        R²      RMSE
0  Random Forest  0.379876  0.216205  1.983205
1        XGBoost  0.383646  0.109689  2.113669
2       CatBoost  0.374203  0.167843  2.043473
3            SVR  0.467401  0.077432  2.151619
4            KNN  0.416127  0.069700  2.160616


In [32]:
data.head()

,Unnamed: 0,Question,Correct_Code,Code_with_Error,Total_Marks,AST_full,AST_predicted_values_Random Forest,AST_predicted_values_XGBoost,AST_predicted_values_CatBoost,AST_predicted_values_SVR,AST_predicted_values_KNN
0,0,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,7.0,CursorKind.FUNCTION_DECL printFactors\n Curso...,7.780476,9.158783,8.083245,5.911610,7.2
1,1,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,8.0,CursorKind.FUNCTION_DECL printFactors\n Curso...,5.739167,6.029593,5.678359,6.269147,5.6
2,2,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\nvoid printFactors(int numb...,5.0,CursorKind.FUNCTION_DECL printFactors\n Curso...,5.064786,5.561528,4.966199,6.273282,7.2
3,3,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\n\nvoid printFactors(int nu...,7.0,CursorKind.FUNCTION_DECL printFactors\n Curso...,7.174000,6.985331,6.514410,6.105001,7.2
4,4,Print the factors of a number,#include <stdio.h>\nvoid printFactors(int numb...,#include <stdio.h>\n\nvoid printFactors(int nu...,5.0,CursorKind.FUNCTION_DECL printFactors\n Curso...,6.045714,6.096070,5.765330,6.329648,5.0
